# 從 World Bank 取得 Buffett indicator

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import os 
import zipfile

In [2]:
# 指定下載檔案存放路徑
download_path = "./temp";

# 檢查路徑是否存在
if os.path.exists(download_path) and os.path.isdir(download_path):
    print(download_path + '資料夾已存在')
else:
    os.makedirs(download_path)
    print('已新增資料夾')

# 設定下載檔案存放路徑
prefs = {"download.default_directory" : download_path}
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs",prefs)

# 將存放路徑清空
for file in os.listdir(download_path):
    file_path = os.path.join(download_path, file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)

./temp資料夾已存在


In [3]:
# 開啟瀏覽器
path = "./chromedriver"
driver = webdriver.Chrome(executable_path=path,options=options)

In [4]:
# 下載檔案(zip file)
url = 'https://data.worldbank.org/indicator/CM.MKT.LCAP.GD.ZS'
driver.get(url)
print("進入網頁!")
button = driver.find_elements_by_link_text('CSV')
print('本程式需時15秒進行下載，請耐心等候...')
button[0].click()
time.sleep(15)
if len(os.listdir(download_path)) > 0:
    print("下載好了!")
    driver.close()
else:
    print('未下載成功，請重新執行本區塊。')

進入網頁!
本程式需時15秒進行下載，請耐心等候...
下載好了!


In [5]:
# 解壓縮
files = os.listdir(download_path)
paths = [os.path.join(download_path,filename) for filename in files]
zip_file = max(paths,key=os.path.getctime)
zip_ref = zipfile.ZipFile(zip_file,'r')
zip_ref.extractall(download_path)
zip_ref.close()

In [6]:
# 讀出檔案中前5筆資料
data_file = zip_file.replace('zip','csv')
df = pd.read_csv(data_file,skiprows=4)
print(df.head())

  Country Name Country Code  \
0        Aruba          ABW   
1  Afghanistan          AFG   
2       Angola          AGO   
3      Albania          ALB   
4      Andorra          AND   

                                      Indicator Name     Indicator Code  1960  \
0  Market capitalization of listed domestic compa...  CM.MKT.LCAP.GD.ZS   NaN   
1  Market capitalization of listed domestic compa...  CM.MKT.LCAP.GD.ZS   NaN   
2  Market capitalization of listed domestic compa...  CM.MKT.LCAP.GD.ZS   NaN   
3  Market capitalization of listed domestic compa...  CM.MKT.LCAP.GD.ZS   NaN   
4  Market capitalization of listed domestic compa...  CM.MKT.LCAP.GD.ZS   NaN   

   1961  1962  1963  1964  1965     ...       2010  2011  2012  2013  2014  \
0   NaN   NaN   NaN   NaN   NaN     ...        NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN   NaN   NaN     ...        NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN   NaN   NaN     ...        NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN 

In [7]:
# 讀取美國單一國家的資料(每年的指標數值)
df.loc[df['Country Code']=='USA']

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
249,United States,USA,Market capitalization of listed domestic compa...,CM.MKT.LCAP.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,115.49734,100.791221,115.555794,143.994423,151.085494,138.336379,146.861593,165.650862,NaN,NaN


In [8]:
# 讀取美國2017年的指標值 (2018尚未有數值)
df[df['Country Code']=='USA']['2017']

249    165.650862
Name: 2017, dtype: float64